In [1]:
import os, xarray as xr, numpy as np, pandas as pd, datetime, dask, random, time, netCDF4
import cartopy.crs as ccrs, matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import cartopy.crs as ccrs, time
import matplotlib.animation as animation
from datetime import datetime as dt, timedelta
import matplotlib.patches as mpatches
dask.config.set({"array.slicing.split_large_chunks": True})

ERROR 1: PROJ: proj_create_from_database: Open of /home/lab_hardik/my_conda/share/proj failed


In [2]:
sa_lon = [60, 98]
sa_lat = [6, 40]

nibox_lon = [68, 78]
nibox_lat = [24, 31]

In [3]:
dir_2d = '/home/scratch/HS_Surface/'

In [4]:
var_lst = ['2m_temperature', 'boundary_layer_height', 'surface_pressure',
                  'convective_available_potential_energy', 'convective_inhibition', # 'mean_sea_level_pressure', 
                  'mean_surface_downward_long_wave_radiation_flux', 'mean_surface_downward_short_wave_radiation_flux', 
                  'mean_surface_net_long_wave_radiation_flux', 'mean_surface_net_short_wave_radiation_flux', 
                  'surface_latent_heat_flux', 'surface_sensible_heat_flux', 
                  'volumetric_soil_water_layer_1']

In [12]:
os.chdir(dir_2d)
var = var_lst[2]

files = [s for s in os.listdir(dir_2d) if s.endswith('.grib') and var in s
#             any([t in s for t in var])
        ]

files = [s for s in files if (int(s.split('_')[-3])>=1980) and (int(s.split('_')[-3])<2010) and 
         (int(s.split('_')[-2]) in [3,4,5,6])]

print(len(files)/122)
files.sort(key = lambda s: [int(s.split('_')[-3]), int(s.split('_')[-2])], reverse=False)
print(files[:5])

30.0
['era5_surface_pressure_1980_3_31.grib', 'era5_surface_pressure_1980_3_29.grib', 'era5_surface_pressure_1980_3_9.grib', 'era5_surface_pressure_1980_3_18.grib', 'era5_surface_pressure_1980_3_21.grib']


In [13]:
var_names_out = [''.join([s[0] for s in var_lst[i].split('_')]) for i in range(len(var_lst))]
var_names_out

['2t',
 'blh',
 'sp',
 'cape',
 'ci',
 'msdlwrf',
 'msdswrf',
 'msnlwrf',
 'msnswrf',
 'slhf',
 'sshf',
 'vswl1']

In [14]:
var_lst[2]

'surface_pressure'

In [19]:
def preprocess_2d_dlymn(ds):
#     print(ds.latitude[0] > ds.latitude[1])
#     print(ds.time.dt.date.max().data)
    if ds.latitude[0] > ds.latitude[1]:
        first=1; last=0
    else:
        first=0; last=1
        
    return ds.sel(
    latitude=slice(sa_lat[first], sa_lat[last]), 
    longitude=slice(sa_lon[0], sa_lon[1])).drop(['number','step']).groupby('time.date').mean()

# var='surface_pressure'
for i in range(len(var_lst)):
    if i==2:
        var = var_lst[i]
        for year in range(1982, 2010):
            print(year)
            files_tmp = [s for s in files if (int(s.split('_')[-3]) == year) and (var in s)]
            print(len(files_tmp) == 122)
            
            if len(files_tmp) < 122:
                print("len(files_tmp) < 122")
                break
            else:
                ds = xr.open_mfdataset(
                    files_tmp, 
                    backend_kwargs = {'indexpath':''},
                    preprocess=preprocess_2d_dlymn,
                #   engine='cfgrib'
                ).sortby(['latitude','longitude'])

                ds['date'] = ds.date.astype('datetime64[ns]')

                var_out = var_names_out[i]
                out_str = 'SA_dly_means_prmnsn_{}_{}.nc'.format(var_out,year)
                dir_out = '/home/data/lab_hardik/data/ERA5/daily_means/2D/'

                print("writing")
                ds.to_netcdf(dir_out + out_str)


1982
True
writing
1983
True
writing
1984
True
writing
1985
True
writing
1986
True
writing
1987
True
writing
1988
True
writing
1989
True
writing
1990
True
writing
1991
True
writing
1992
True
writing
1993
True
writing
1994
True
writing
1995
True
writing
1996
True
writing
1997
True
writing
1998
True
writing
1999
True
writing
2000
True
writing
2001
True
writing
2002
True
writing
2003
True
writing
2004
True
writing
2005
True
writing
2006
True
writing
2007
True
writing
2008
True
writing
2009
True
writing
